In [ ]:
# Installing required packages
#install.packages("")

In [38]:
# Libraries
library(xts)
library(corrplot)
library(dplyr)
library(dotenv)

In [42]:
# Load paths
setwd('..')
load_dot_env()
trends_path <- Sys.getenv("GOOGLE_TRENDS")
crypto_path <- Sys.getenv("CRYPTO")
data_path <- Sys.getenv("DATA_PATH")
my_root <- Sys.getenv("MY_ROOT")

Warning message in readLines(file):
"incomplete final line found on '.env'"

# Loading data base

In [43]:
setwd(data_path)

# The database contains returns of btc price and google trends info.
db <- read.table("data_base.csv", sep=",", header = TRUE)

# Exploratory analysis

In [44]:
str(db)

'data.frame':	29244 obs. of  63 variables:
 $ date                  : Factor w/ 29240 levels "2017-08-27 09:00:00",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ altcoins.season       : int  0 0 0 0 0 0 0 0 0 0 ...
 $ banking.system        : int  29 28 24 25 24 24 18 19 13 15 ...
 $ bce                   : int  34 34 39 41 41 41 47 44 47 49 ...
 $ bill.gates            : int  36 39 39 39 38 41 37 43 41 46 ...
 $ bitcoin.alert         : int  64 25 45 39 7 41 14 24 9 29 ...
 $ bitcoin.bubble        : int  7 13 11 10 16 19 17 22 25 30 ...
 $ bitcoin.dump          : int  29 0 0 0 0 30 32 48 80 14 ...
 $ bitcoin.hack          : int  56 45 55 39 46 43 60 48 42 35 ...
 $ bitcoin.pump          : int  75 8 16 42 8 9 28 21 92 25 ...
 $ bitcoin.whales        : int  23 0 0 0 0 0 26 0 0 0 ...
 $ bitcoin               : int  56 56 56 58 57 58 60 62 60 61 ...
 $ blockchain            : int  58 57 55 52 54 57 51 55 54 55 ...
 $ coindesk              : int  35 34 36 29 35 37 35 40 44 50 ...
 $ coingecko             :

In [45]:
# data_base (Google trends and btc returns) dimensions:
print(paste("Rows: ", dim(db)[1], "Columns: ", dim(db)[2]))

[1] "Rows:  29244 Columns:  63"


## Is there any NA

In [46]:
any(is.na(db))

[1] TRUE

In [47]:
# Getting the columns names with NA's
cols_nas <- colnames(db)[colSums(is.na(db)) > 0]
sort(cols_nas)
print(paste("Amount of columns with NA: ", length(cols_nas)))

[1] "altcoins.season"        "amazon"                 "apple"                 
 [4] "banking.system"         "bce"                    "bill.gates"            
 [7] "bitcoin"                "bitcoin.alert"          "bitcoin.bubble"        
[10] "bitcoin.dump"           "bitcoin.hack"           "bitcoin.pump"          
[13] "bitcoin.twitter"        "bitcoin.whales"         "blockchain"            
[16] "bloomberg"              "china.economy"          "coindesk"              
[19] "coingecko"              "coinmarketcap"          "cointelegraph"         
[22] "crypto.assets"          "cryptonews"             "debt"                  
[25] "debt.jubilee"           "derivatives.market"     "dollar"                
[28] "economic.collapse"      "economic.stimulus"      "economy.recovery"      
[31] "elon.musk"              "ethereum"               "facebook"              
[34] "fed"                    "financial.bubble"       "financial.crisis"      
[37] "financial.independence" "housing.market"         "investing"             
[40] "jamie.dimon"            "jerome.powell"          "litecoin"              
[43] "microsoft"              "money.printing"         "netflix"               
[46] "ray.dalio"              "real.economy"           "real.estate"           
[49] "repo.market"            "returns"                "ripple"                
[52] "satoshi.nakamoto"       "sec"                    "smart.contracts"       
[55] "stimulus.bill"          "stocks.bubble"          "stocks.crash"          
[58] "twitter"                "vitalik.buterin"        "volatility"            
[61] "warren.buffett"         "wonder.woman"

[1] "Amount of columns with NA:  62"


>**As we can see, all the columns have na's within, let's compare it with the google trends data base**

In [48]:
setwd(trends_path)
# This database contains only google trends info 
gt <- read.table("google trends.csv", sep=",", header = TRUE)

In [49]:
# Google trends data base dimensions:
print(paste("Rows: ", dim(gt)[1], "Columns: ", dim(gt)[2]))

[1] "Rows:  29197 Columns:  62"


In [50]:
# Is there any NA
any(is.na(gt))

[1] TRUE

In [51]:
# Compare the columns with NA's before joining with the btc prices (returns)
cols_nas_gt <- colnames(gt)[colSums(is.na(gt)) > 0]
sort(cols_nas_gt)
print(paste("Amount of columns with NA: ", length(cols_nas_gt)))

[1] "altcoins.season"  "bill.gates"       "bloomberg"        "china.economy"   
 [5] "coindesk"         "cryptonews"       "debt"             "economy.recovery"
 [9] "elon.musk"        "facebook"         "financial.crisis" "investing"       
[13] "jamie.dimon"      "jerome.powell"    "litecoin"         "microsoft"       
[17] "ray.dalio"        "real.economy"     "satoshi.nakamoto" "sec"             
[21] "stimulus.bill"    "vitalik.buterin"  "warren.buffett"   "wonder.woman"

[1] "Amount of columns with NA:  24"


>**When we joined the google trends and the btc_price data bases, something causes passing from 24 columns with na's to all of the 62 columns with na's**

In [52]:
# Check in google trends data base the columns with no ns's withing
vect_temp_columns <- setdiff(cols_nas, cols_nas_gt)
sort(vect_temp_columns)
print(paste("Number of columns with no na's before joining the returns columns: " , length(vect_temp_columns)))

[1] "amazon"                 "apple"                  "banking.system"        
 [4] "bce"                    "bitcoin"                "bitcoin.alert"         
 [7] "bitcoin.bubble"         "bitcoin.dump"           "bitcoin.hack"          
[10] "bitcoin.pump"           "bitcoin.twitter"        "bitcoin.whales"        
[13] "blockchain"             "coingecko"              "coinmarketcap"         
[16] "cointelegraph"          "crypto.assets"          "debt.jubilee"          
[19] "derivatives.market"     "dollar"                 "economic.collapse"     
[22] "economic.stimulus"      "ethereum"               "fed"                   
[25] "financial.bubble"       "financial.independence" "housing.market"        
[28] "money.printing"         "netflix"                "real.estate"           
[31] "repo.market"            "returns"                "ripple"                
[34] "smart.contracts"        "stocks.bubble"          "stocks.crash"          
[37] "twitter"                "volatility"

[1] "Number of columns with no na's before joining the returns columns:  38"


>**For simplicity let's focus (for now) just in one column, the amazon column:**

In [53]:
# Let's check out a few of rows with na's withing
index_cols_nas <- which(is.na(db$amazon))
index_cols_nas <- index_cols_nas[1:10]
index_cols_nas

[1] 3981 3983 3985 3987 3989 3991 3993 3995 3997 3999

In [54]:
db[3971:3997,1:63]

,date,altcoins.season,banking.system,bce,bill.gates,bitcoin.alert,bitcoin.bubble,bitcoin.dump,bitcoin.hack,bitcoin.pump,...,ripple,netflix,sec,investing,bloomberg,china.economy,facebook,fed,financial.crisis,returns
3971,2018-02-08 18:00:00,0,39,52,36,31,20,22,65,30,...,36,70,64,49,44,64,97,45,0,NA
3972,2018-02-08 19:00:00,0,39,56,39,30,13,38,49,47,...,34,69,65,48,45,68,94,39,0,NA
3973,2018-02-08 20:00:00,0,54,55,39,16,13,22,65,44,...,35,66,69,51,47,85,88,38,0,NA
3974,2018-02-08 21:00:00,0,84,47,36,40,10,17,57,46,...,37,59,73,54,51,78,82,36,0,NA
3975,2018-02-08 22:00:00,0,91,44,33,60,12,30,66,43,...,40,50,78,59,52,91,81,30,47,NA
3976,2018-02-08 23:00:00,0,87,36,31,34,19,24,69,39,...,43,39,88,63,65,75,81,27,47,NA
3977,2018-02-09 00:00:00,0,91,43,27,47,17,0,61,4,...,43,28,86,72,71,61,80,26,43,NA
3978,2018-02-09 01:00:00,0,80,52,24,14,14,10,43,24,...,42,24,74,82,67,55,78,24,0,NA
3979,2018-02-09 02:00:00,75,66,67,22,13,9,18,79,26,...,40,23,66,81,60,57,77,24,36,NA
3980,2018-02-09 03:00:00,0,74,70,24,18,9,9,35,17,...,36,23,62,67,53,40,78,24,0,NA


>**Something strange happened with the date format, it should be as 2018-02-09 11:00:00 but instead of having 00:00 minute format, it has a 28:14 minute format.**
That is, the date format at google trends data base is correct, however, the date format at btc_prices is wrong.
~~~
So, let's fix this problem with na's:
~~~

In [55]:
twentyeight_problem <- function(value){
  word <- as.character(value)
  word <- sub(":28:.*", ":00:00", word)
  return(word)
}

In [56]:
db_a <- db
db_a$date <- sapply(db_a$date, twentyeight_problem)

In [57]:
index_duplicated <- which(duplicated(db_a$date))


for(i in index_duplicated){
    return_i <- db_a$returns[i]
    db_a$returns[i-1] = return_i
}

db_a <- db_a[!duplicated(db_a$date), ]


In [58]:
db_a[3971:3997,1:63]

,date,altcoins.season,banking.system,bce,bill.gates,bitcoin.alert,bitcoin.bubble,bitcoin.dump,bitcoin.hack,bitcoin.pump,...,ripple,netflix,sec,investing,bloomberg,china.economy,facebook,fed,financial.crisis,returns
3972,2018-02-08 19:00:00,0,39,56,39,30,13,38,49,47,...,34,69,65,48,45,68,94,39,0,NA
3973,2018-02-08 20:00:00,0,54,55,39,16,13,22,65,44,...,35,66,69,51,47,85,88,38,0,NA
3974,2018-02-08 21:00:00,0,84,47,36,40,10,17,57,46,...,37,59,73,54,51,78,82,36,0,NA
3975,2018-02-08 22:00:00,0,91,44,33,60,12,30,66,43,...,40,50,78,59,52,91,81,30,47,NA
3976,2018-02-08 23:00:00,0,87,36,31,34,19,24,69,39,...,43,39,88,63,65,75,81,27,47,NA
3977,2018-02-09 00:00:00,0,91,43,27,47,17,0,61,4,...,43,28,86,72,71,61,80,26,43,NA
3978,2018-02-09 01:00:00,0,80,52,24,14,14,10,43,24,...,42,24,74,82,67,55,78,24,0,NA
3979,2018-02-09 02:00:00,75,66,67,22,13,9,18,79,26,...,40,23,66,81,60,57,77,24,36,NA
3980,2018-02-09 03:00:00,0,74,70,24,18,9,9,35,17,...,36,23,62,67,53,40,78,24,0,5.09351209
3982,2018-02-09 04:00:00,0,65,69,22,35,7,12,57,23,...,34,25,60,66,48,47,81,24,0,1.47313734


>**As we can see, the problem was fixed, at least with the amazon column. Yet, the problem was fixed for all the columns thanks the function we wrote.**


In [59]:
# Let's check if there is columns with na's.
cols_nas_dba <- colnames(db_a)[colSums(is.na(db_a)) > 0]
sort(cols_nas_dba)
print(paste("Amount of columns with NA's: ", length(cols_nas_dba)))

[1] "altcoins.season"  "bill.gates"       "bloomberg"        "china.economy"   
 [5] "coindesk"         "cryptonews"       "debt"             "economy.recovery"
 [9] "elon.musk"        "facebook"         "financial.crisis" "investing"       
[13] "jamie.dimon"      "jerome.powell"    "litecoin"         "microsoft"       
[17] "ray.dalio"        "real.economy"     "returns"          "satoshi.nakamoto"
[21] "sec"              "stimulus.bill"    "vitalik.buterin"  "warren.buffett"  
[25] "wonder.woman"

[1] "Amount of columns with NA's:  25"


>**The last code show us one new column with na's, that is the returns column.**

In [60]:
index_nas_r_a <- which(is.na(db_a$returns))
index_nas_r_a
print(paste("Amount of NA's within returns column: ", length(index_nas_r_a)))

[1]   243   244   245   246   247   248   249  3110  3947  3948  3949  3950
 [13]  3951  3952  3953  3954  3955  3956  3957  3958  3959  3960  3961  3962
 [25]  3963  3964  3965  3966  3967  3968  3969  3970  3971  3972  3973  3974
 [37]  3975  3976  3977  3978  7260  7261  7262  7263  7264  7265  7266  7267
 [49]  7268  7269  7295  7451  7452  7453  7454  7455  7456  7457 10024 10025
 [61] 10026 10643 10644 10645 10646 10647 10648 10649 13475 13476 13477 13478
 [73] 13479 13480 15012 15013 15014 15015 15016 15017 15018 15019 15020 15021
 [85] 17219 17220 17221 17222 17223 17224 17225 17226 19378 19379 19666 19667
 [97] 21490 21740 21741 21742 21743 21744 22074 23314 23315 24850 24851 24852
[109] 28573 29086 29087 29088 29169

[1] "Amount of NA's within returns column:  113"


In [61]:
# Finally we get rid of the na's.
db_a <- na.omit(db_a)

In [62]:
# Is there any NA
any(is.na(db_a))

[1] FALSE

In [63]:
# New data_base (Google trends and btc returns) dimensions:
print(paste("Rows: ", dim(db_a)[1], "Columns: ", dim(db_a)[2]))

[1] "Rows:  24669 Columns:  63"


In [64]:
setwd(data_path)
# Save all the changes:
write.csv(db_a, file = "data_base.csv", row.names = FALSE)